In [ ]:
# Mount your Google Drive if needed

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install transformers[torch]==4.31.0
!pip install transformers
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Note: Some libraries or dependencies used in this code may become deprecated over time.
# If you encounter deprecated libraries, replace them with suitable alternatives and
# update the corresponding sections of the code that rely on the deprecated classes or methods accordingly.

import os
import json
import gzip
import pandas as pd
import collections
import nltk
from google.colab import files

from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import pickle
from IPython.display import clear_output
import torch
from transformers import RobertaTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,  recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold
from transformers import  Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification
# from datasets import load_metric
import evaluate


In [ ]:
if torch.cuda.is_available():
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))

1
Tesla T4


# Training

In [ ]:

def set_seed(SEED):
    """ Set random seed to all """
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True



class AmazonDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels= None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
           item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])




def prepare_auxilary_datasets(dataset):

    X_test =  dataset["sentence"]
    y_test =  dataset["label"]   if "label" in dataset.columns.values else   dataset["helpful"]
    # Encoding
    test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=40)
    test_dataset = AmazonDataset(test_encodings, y_test.tolist())
    return test_dataset


def compute_metrics(eval_pred):

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    recall = recall_score(y_true=labels, y_pred=predictions)
    precision = precision_score(y_true=labels, y_pred=predictions)
    f1 = f1_score(y_true=labels, y_pred=predictions)
    auc = roc_auc_score(y_true= labels, y_score= logits[:, 1])

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'AUC': auc
    }




def return_training_args():

    training_args = TrainingArguments(
        output_dir="test-amazon",
        #evaluation_strategy = "epoch",  # there would be no evaluation during training when doing simple K-fold cross validation a
        learning_rate=3e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        max_steps = 1500,
        weight_decay=0.01,
        load_best_model_at_end=False,
        logging_strategy = "steps",
        logging_steps = 100,
        save_strategy= "no",
    )
    return training_args




In [ ]:
def data_exclude_func(dataset, setting, rate):
    sentence_ids = []
    selected_needs=[]
    forbidden_need_ids = []
    if setting == 'LF_testing':
       for k, v in frequency.items():
           if (v[0] <= rate):
              sentence_ids.append(v[1])
              selected_needs.append(k)
           else:
              forbidden_need_ids.append(k)
    elif setting == 'HF_testing':
       for k, v in frequency.items():
           if (v[0] >= rate):
              sentence_ids.append(v[1])
              selected_needs.append(k)
           else:
              forbidden_need_ids.append(k)
    exclude_ids = list(set([id for id_list in sentence_ids for id in id_list]))
    df_excluded_primary = dataset[dataset['Sentence_ID'].isin(exclude_ids)]
    # Rows with sentence_id not in exclude_ids will go to df
    df_main = dataset[~dataset['Sentence_ID'].isin(exclude_ids)]
    # Check if any forbidden need ID is present in the specified columns
    condition = df_excluded_primary[['Need_ID_1', 'Need_ID_2', 'Need_ID_3', 'Need_ID_4', 'Need_ID_5',
                                 'Need_ID_6', 'Need_ID_7', 'Need_ID_8', 'Need_ID_9', 'Need_ID_10']].isin(forbidden_need_ids).any(axis=1)
    # Exclude rows that meet the condition
    df_garbage = df_excluded_primary[condition]
    df_excluded = df_excluded_primary[~condition]

    return df_main, df_excluded, df_garbage



##################### testing data_exclude_func  ###################

data_set_1 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/8000_sentences_with_need_IDs.csv')

# Create a dictionary from `data_set_1` that includes the population count for each customer need.
frequency = pd.read_pickle('/content/drive/MyDrive/paper1/datasets/dictionary frequency of occurence of needs')

ratios_maxstep = {
    'LF_testing': [  # Setting A
        (5, 2400),(10, 2400),(20, 2400),  # first value is ratio, and the other is max_step
        (30, 2200),(40, 2200 ),
        (50, 2000),(75, 1600),
        (175, 1400),(190, 1200),
        (200, 1000),(210, 800),
        (300, 500),(562, 500)
          ],  # LF_testing: Frequent Training

    'HF_testing': [ # Setting B
        (350, 2200),(320, 2000),
         (290, 1800),(210, 1600),
          (207, 1600),(205, 1500),
           (195, 1500),(175, 1200),
            (110, 1000),(75, 800),
             (58, 600),(43, 500)
             ]  # HF_testing: Infrequent Training
}


# The following provides the sample population for each experimental setting.
# This offers an initial estimate of the training time required.

for setting, rate_list in ratios_maxstep.items():
    print(f'################################### {setting} ###############################')
    for ratio in rate_list:
        x, y, z= data_exclude_func(data_set_1, setting, rate = ratio[0])
        if x[x.label==1].shape[0] < x[x.label==0].shape[0]:
           training_size = 0.8 * (2 * x[x.label==1].shape[0] )
           test_size_exc = y.shape[0]
           test_size_gb = z.shape[0]
        else:
           training_size = 0.8 * (x[x.label==1].shape[0] + x[x.label==0].shape[0] )
           test_size_exc = y.shape[0]
           test_size_gb = z.shape[0]
          #  print(f'{setting} and {ratio[0]}')
        print(f'{ratio[0]}: {training_size}, {test_size_exc}, {test_size_gb}')


################################### LF_testing ###############################
5: 6360.0, 36, 14
10: 6287.200000000001, 100, 41
20: 6238.400000000001, 141, 61
30: 5952.0, 329, 132
40: 5739.200000000001, 406, 188
50: 5278.400000000001, 604, 278
75: 4372.8, 1027, 421
175: 3384.0, 1628, 438
190: 2652.8, 2098, 425
200: 2209.6, 2380, 420
210: 1756.8000000000002, 2740, 343
300: 1038.4, 3282, 250
562: 702.4000000000001, 3618, 124
################################### HF_testing ###############################
350: 5788.8, 439, 124
320: 5251.200000000001, 649, 250
290: 4795.2, 898, 286
210: 4384.0, 1098, 343
207: 4096.0, 1207, 414
205: 3808.0, 1381, 420
195: 3606.4, 1545, 382
175: 2604.8, 2115, 438
110: 2222.4, 2378, 414
75: 1643.2, 2733, 421
58: 1027.2, 3195, 344
43: 702.4000000000001, 3548, 194


In [ ]:
frequency = pd.read_pickle('/content/drive/MyDrive/paper1/datasets/dictionary frequency of occurence of needs')

# This experiment is specific to the Oral-Care dataset.
# If you do not need to evaluate the cross-domain performance of models on other domains,
# please comment out datasets 2-5.
data_set_1 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/8000_sentences_with_need_IDs.csv')
data_set_2 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/Electronics.csv')
data_set_3 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/baby.csv')
data_set_4 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/Pet_supplies.csv')
data_set_5 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/Sport_outdoors.csv')

# Ensure the 'sentence' column is of type 'str'
data_set_1['sentence'] = data_set_1['sentence'].astype(str)
data_set_2['sentence'] = data_set_2['sentence'].astype(str)
data_set_3['sentence'] = data_set_3['sentence'].astype(str)
data_set_4['sentence'] = data_set_4['sentence'].astype(str)
data_set_5['sentence'] = data_set_5['sentence'].astype(str)

In [ ]:
models = {
              'albert_base' :"albert-base-v2",
              'distilbert'  :"distilbert-base-uncased",
              'bertB'       :"bert-base-uncased",
              'bertL'       :"bert-large-uncased",
              'robertB'     :"roberta-base",
              'robertL'     :"roberta-large",
              'allenaiB'    :"allenai/reviews_roberta_base",

              'ALBERT-L'      :"albert/albert-large-v2",
              'DISRoBERTa-B'  :"distilbert/distilroberta-base",
              'XLNet-B'       :"xlnet/xlnet-base-cased",
              'XLNet-L'       :"xlnet/xlnet-large-cased",
              'DEBERT-B'      :"microsoft/deberta-base",
              'DEBERT-L'      :"microsoft/deberta-large",
              'XLM-B'         :"FacebookAI/xlm-roberta-base",
              'XLM-L'         :"FacebookAI/xlm-roberta-large"
          }

# Full training with the following settings requires over 100 hours on an A100 GPU.
ratios_maxstep = {
    'LF_testing': [
        (5, 2400), (10, 2400), (20, 2400),   # first value is ratio, and the other is max_step
        (30, 2200), (40, 2200 ),(50, 1900), (300, 600), (562, 500),
        (75, 1600), (175, 1400), (190, 1200),(200, 1000), (210, 800)

          ]  # LF_testing: Frequent Training

    'HF_testing': [
        (350, 2200),(320, 2000),
        (290, 1800),(210, 1600), (58, 500),

          (207, 1600),(205, 1500),
          (195, 1500),(175, 1200), (110, 1000)

        (75, 800), (43, 500)
             ]  # HF_testing: Infrequent Training
}


seeds = [
    9741, 1694, 6932, 94, 791, 5, 1759, 323, 200, 999
    ]  # randomly selected
all_datasets_ratio_models_seed_results = {}
results = []
results_exclude = []
results_garbage = []
results_uninf_exc = []


setting_ratio = 'HF_testing'
for ratio_max_step in ratios_maxstep[setting_ratio]:
    training_args = return_training_args()  # must be removed when adding in the loop
    ratio = ratio_max_step[0]
    max_stepp =  ratio_max_step[1]
    training_args.max_steps = max_stepp
    if max_stepp < 700 :
       training_args.logging_steps = 50
    else:
       training_args.logging_steps = 100

    all_datasets_ratio_models_seed_results.update({ratio:{}})
    for model_name, model_address in models.items():
        tokenizer = AutoTokenizer.from_pretrained(model_address)
        all_datasets_ratio_models_seed_results[ratio].update({model_name:{}})
        if model_name == 'bertL':
           training_args.learning_rate = 2e-5
        if model_name == 'robertL':
           training_args.learning_rate = 1.25e-5
        if model_name == 'ALBERT-L' :
           training_args.learning_rate = 8e-6
        if model_name == 'DISRoBERTa-B':
           training_args.learning_rate = 2e-5
        if model_name == 'XLNet-B':
           training_args.learning_rate = 2e-5

        if model_name == 'XLNet-L' :
           training_args.learning_rate = 1e-5
           training_args.per_device_train_batch_size = 32
           max_stepp = int(max_stepp * 3 / 4)
           training_args.max_steps = max_stepp

        if model_name == 'DEBERT-B':
           training_args.learning_rate = 3e-5
           training_args.per_device_train_batch_size = 16
           max_stepp =  ratio_max_step[1]
           training_args.max_steps = max_stepp

        if model_name == 'DEBERT-L':
           training_args.learning_rate = 1e-5
        if model_name ==  'XLM-B':
           training_args.learning_rate = 2e-5
        if model_name ==  'XLM-L':
           training_args.learning_rate = 8e-6

        for seed in seeds:
            set_seed(seed)
            training_args.seed = seed
            print(ratio, model_name, training_args.seed , training_args.learning_rate, training_args.max_steps)


            df_main, df_exclude, df_garbage = data_exclude_func(data_set_1,
                                                                setting= setting_ratio,
                                                                rate= ratio
                                                                )

            # for ratio 5, 10, and 20 for LF_testing setting, this balance startegy does not work as
            # the number of samples in the uninf. part is already less than inf. part
            # thus we add the follwoing if statement to preserve the consitency in the code
            # and solve the issue.
            if df_main[df_main.label==1].shape[0] < df_main[df_main.label==0].shape[0]:
               balance = True
            else:
               balance = False

            if balance == True:
                drop_uninf = abs(df_main[df_main.label == 1].shape[0] - df_main[df_main.label == 0].shape[0] )
                df_uninf_exc = df_main[df_main.label == 0].sample(n= int(drop_uninf), random_state = seed)
                df_main = df_main.drop(df_uninf_exc.index)
                test_uninf_exc_encodings = tokenizer(df_uninf_exc.sentence.tolist(), truncation=True, padding=True, max_length=40)
                test_uninf_dataset = AmazonDataset(test_uninf_exc_encodings)

            df_train = df_main.sample(frac = 0.8, random_state = seed)
            # Find the rows not selected
            df_test = df_main.drop(df_train.index)

            # Encoding
            train_main_encodings = tokenizer(df_train.sentence.tolist(), truncation=True, padding=True, max_length=40)
            test_main_encodings = tokenizer(df_test.sentence.tolist(), truncation=True, padding=True, max_length=40)
            test_exclude_encodings = tokenizer(df_exclude.sentence.tolist(), truncation=True, padding=True, max_length=40)
            test_garbage_encodings = tokenizer(df_garbage.sentence.tolist(), truncation=True, padding=True, max_length=40)


            train_dataset = AmazonDataset(train_main_encodings, df_train.label.tolist())
            test_main_dataset = AmazonDataset(test_main_encodings, df_test.label.tolist())
            test_exclude_dataset = AmazonDataset(test_exclude_encodings)
            test_garbage_dataset = AmazonDataset(test_garbage_encodings)


            print(f'Model seed is: {training_args.seed}, total, training informative and uninf samples: {df_main.shape[0]} ,\
             {df_train[df_train.label == 1].shape[0]}, {df_train[df_train.label == 0].shape[0]}')
            model = AutoModelForSequenceClassification.from_pretrained(model_address) # option 1: allenai/reviews_roberta_base  # option2: roberta-base
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                #eval_dataset=val_dataset,             # evaluation dataset must be off when doing simple K-fold cross validation
                # callbacks= [EarlyStoppingCallback(early_stopping_patience= 3,early_stopping_threshold=0.1)], # does not properly work
                compute_metrics=compute_metrics
            )

            trainoutput = trainer.train()
            tr_loss = trainoutput.training_loss
            pred = trainer.predict(test_main_dataset)
            results.append(["skin-care", ratio, model_name, seed, pred.metrics['test_accuracy'], pred.metrics['test_f1'], pred.metrics['test_precision'], pred.metrics['test_recall'], pred.metrics['test_AUC']])

            # raw predictions for unseen data
            raw_pred, _, _ = trainer.predict(test_exclude_dataset)
            # Preprocess raw predictions
            y_prediction = np.argmax(raw_pred, axis=1)
            recall_exc = recall_score(y_true= df_exclude.label, y_pred= y_prediction)
            results_exclude.append(["skin-care-exc", ratio, model_name, seed, y_prediction.shape[0], recall_exc])

            # raw predictions for garbage data
            raw_pred_garbage, _, _ = trainer.predict( test_garbage_dataset)
            # Preprocess raw predictions
            y_prediction_garbage = np.argmax(raw_pred_garbage, axis=1)
            recall_garbage = recall_score(y_true= df_garbage.label, y_pred= y_prediction_garbage)
            results_garbage.append(["skin-care-garbage", ratio, model_name, seed, y_prediction_garbage.shape[0], recall_garbage])

            if balance == True:
               # raw predictions for uninf excluded data
               raw_pred_uninf_exc, _, _ = trainer.predict( test_uninf_dataset)
               # Preprocess raw predictions
               y_prediction_uninf_exc = np.argmax(raw_pred_uninf_exc, axis=1)
               recall_uninf_exc = recall_score(y_true= df_uninf_exc.label, y_pred= y_prediction_uninf_exc, pos_label= 0)
               results_uninf_exc.append(["skin-care-uninf_exc", ratio, model_name, seed, y_prediction_uninf_exc.shape[0], recall_uninf_exc])

            # If you comment out datasets 2-5, make sure to also remove their corresponding keys
            # from the `all_datasets_ratio_models_seed_results` dictionary.
            all_datasets_ratio_models_seed_results[ratio][model_name].update(
                                                                           { seed : {
                                                                                            "skin-care":            [df_train.Sentence_ID.values, df_test.Sentence_ID.values, pred, tr_loss, 0],
                                                                                            "electronics":          trainer.predict(prepare_auxilary_datasets(data_set_2)),
                                                                                            "baby":                 trainer.predict(prepare_auxilary_datasets(data_set_3)),
                                                                                            "pet-supplies":         trainer.predict(prepare_auxilary_datasets(data_set_4)),
                                                                                            "Sport-outdoors":       trainer.predict(prepare_auxilary_datasets(data_set_5)),
                                                                                            "skin-care-EXC":        [df_exclude.Sentence_ID.values, raw_pred],
                                                                                            "skin-care-GB":         [df_garbage.Sentence_ID.values, raw_pred_garbage],
                                                                                            "skin-care-uninf_exc":  [df_uninf_exc.Sentence_ID.values, raw_pred_uninf_exc] if balance  else [],
                                                                                          }
                                                                           })

            raw_pred, _, _ = trainer.predict(test_exclude_dataset)

            print(f'for ratio of {ratio}, model of {model_name}, seed of {seed} performance is:\n', recall_exc," and ",pred.metrics)

    print('number of training samples: ', df_train.shape)

    # Checkpointing
    # Specify the path in Google Drive
    file_path = f'/content/drive/MyDrive/paper1/all_results_ratio_{ratio}'
    # Serialize and save the dictionary
    with open(file_path, 'wb') as handle:
         pickle.dump(all_datasets_ratio_models_seed_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_result = pd.DataFrame(results, columns = ['category', 'ratio' ,'model', 'seed', 'accuracy','f1-score', 'precision', 'recall', 'AUC'])
display(df_result)
display(df_result.groupby(df_result.category).mean(numeric_only =True))


df_result_exc = pd.DataFrame(results_exclude, columns = ['category', 'ratio' ,'model', 'seed', 'NOS', 'recall'])
display(df_result_exc)
display(df_result_exc.groupby(df_result_exc.category).mean(numeric_only =True))


df_result_gb = pd.DataFrame(results_garbage, columns = ['category', 'ratio' ,'model', 'seed', 'NOS', 'recall'])
display(df_result_gb)
display(df_result_gb.groupby(df_result_gb.category).mean(numeric_only =True))

# If you comment out datasets 2-5, make sure to also remove their corresponding keys
# from the 'tables` dictionary.
tables = {"skin-care": results,"electronics": [],"baby": [],
          "pet-supplies": [],"Sport-outdoors": []}

for rto, values in all_datasets_ratio_models_seed_results.items():
    for model_name, value in values.items():
        for seed, val in value.items():
            for dataset, v in val.items():
                if dataset == "skin-care":
                  tables["skin-care"] =  df_result
                elif (dataset != "skin-care-EXC") and (dataset != "skin-care-GB") and (dataset != "skin-care-uninf_exc"):
                   tables[dataset].append([dataset, rto, model_name, seed, v.metrics['test_accuracy'],
                                           v.metrics['test_f1'], v.metrics['test_precision'],
                                           v.metrics['test_recall'], v.metrics['test_AUC']])

df_total_list = [pd.DataFrame(t, columns = ['category','ratio' ,'model', 'seed', 'accuracy','f1-score', 'precision', 'recall', 'AUC']) for k, t in tables.items()]
df = pd.concat(df_total_list).drop_duplicates().reset_index(drop=True)
display(df.groupby(df.category).mean(numeric_only =True))




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

...

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet/xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,0.690100
200,0.561400
300,0.460900
400,0.361900
500,0.257600
600,0.184600
700,0.136100


for ratio of 110, model of XLNet-L, seed of 9741 performance is:
 0.5794785534062237  and  {'test_loss': 0.8476578593254089, 'test_accuracy': 0.7589928057553957, 'test_f1': 0.7774086378737541, 'test_precision': 0.73125, 'test_recall': 0.8297872340425532, 'test_AUC': 0.853742817207641, 'test_runtime': 1.1769, 'test_samples_per_second': 472.416, 'test_steps_per_second': 7.647}
110 XLNet-L 1694 1e-05 750
Model seed is: 1694, total, training informative and uninf samples: 2778 ,             1100, 1122


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet/xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [ ]:

############### google drive  ########3
path_df = f'/content/drive/MyDrive/paper1/df_result'
# Save the DataFrame as a pickle file
df_result.to_pickle(path_df)

path_df = f'/content/drive/MyDrive/paper1/df_result_exc'
# Save the DataFrame as a pickle file
df_result_exc.to_pickle(path_df)

path_df = f'/content/drive/MyDrive/paper1/df_result_gb'
# Save the DataFrame as a pickle file
df_result_gb.to_pickle(path_df)



# Specify the path in Google Drive
file_path = f'/content/drive/MyDrive/paper1/all_datasets_seed_fold_results'
# Serialize and save the dictionary
with open(file_path, 'wb') as handle:
    pickle.dump(all_datasets_ratio_models_seed_results, handle, protocol=pickle.HIGHEST_PROTOCOL)